# Demo: WizardLM, WizardMath, and WizardCoder with MLC LLM

WizardLM recently released their WizardMath model, which has achieved impressive results on various benchmarks. We take this opportunity to demonstrate MLC LLM's support for the Wizard model family: https://github.com/nlpxucan/WizardLM.

Specifically, we will look at:
- **[WizardLM](https://github.com/nlpxucan/WizardLM/tree/main/WizardLM): an instruction-following LLM using Evol-Instruct**
- **[WizardCoder](https://github.com/nlpxucan/WizardLM/tree/main/WizardCoder): a code LLM with Evol-Instruct**
- **[WizardMath](https://github.com/nlpxucan/WizardLM/tree/main/WizardMath): a mathematical reasoning LLM via reinforced Evol-Instruct**

The task-specific Wizard models resonate with one of the visions of MLC LLM: deploying LLMs natively, each acting as **a personal AI assistant for a specific realm of tasks**.

In this notebook, we walk over the steps of using MLC LLM to run these pre-compiled Wizard models! We have uploaded various versions of the pre-compiled and quantized Wizard models here: https://huggingface.co/mlc-ai.

Learn more about MLC LLM here: https://mlc.ai/mlc-llm/docs.

Click the button below to get started!

<a target="_blank" href="https://colab.research.google.com/github/mlc-ai/notebooks/blob/main/mlc-llm/models/demo_WizardLM_Math_Coder.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Step 0. Install MLC LLM

We will start from setting up the environment. First, let us create a new Conda environment, in which we will run the rest of the notebook.

```
conda create --name mlc-llm python=3.10
conda activate mlc-llm
```

**Google Colab**

- If you are running this in a Google Colab notebook, you would not need to create a conda environment.
- However, be sure to change your runtime to GPU by going to `Runtime` > `Change runtime type` and setting the Hardware accelerator to be "GPU".

If you are using CUDA, you can run the following command to confirm that CUDA is set up correctly, and check the driver version number as well as what GPUs are currently available for use.



In [ ]:
!nvidia-smi

Next, let's download the MLC-AI and MLC-Chat nightly build packages. If you are running in a Colab environment, then you can just run the following command. Otherwise, go to https://mlc.ai/package/ and replace the command below with the one that is appropriate for your hardware and OS.

**Google Colab**: If you are using Colab, you may see the red warnings such as "You must restart the runtime in order to use newly installed versions." For our purpose, we can disregard them, the notebook will still run correctly.

In [ ]:
!pip install --pre --force-reinstall mlc-ai-nightly-cu118 mlc-chat-nightly-cu118 -f https://mlc.ai/wheels

Let's confirm we have installed the packages successfully!

In [ ]:
!python -c "import tvm; print('tvm installed properly!')"
!python -c "import mlc_chat; print('mlc_chat installed properly!')"

## Step 1. Download Prebuilt Models and Library

These commands will download many prebuilt libraries (e.g. `.so` files) as well as the precompiled Wizard models. This may take a while. If in **Google Colab**, you can verify that the files are being downloaded by clicking on the folder icon on the left.

Note: If you are NOT running in **Google Colab** you may need to run this line `!conda install git git-lfs` to install `git` and `git-lfs` before running the following cell.

In [ ]:
!git lfs install

In [ ]:
!mkdir -p dist/prebuilt
!git clone https://github.com/mlc-ai/binary-mlc-llm-libs.git dist/prebuilt/lib

In [ ]:
!cd dist/prebuilt && git clone https://huggingface.co/mlc-ai/mlc-chat-WizardCoder-15B-V1.0-q4f16_1

In [ ]:
!cd dist/prebuilt && git clone https://huggingface.co/mlc-ai/mlc-chat-WizardLM-13B-V1.2-q4f16_1

In [ ]:
!cd dist/prebuilt && git clone https://huggingface.co/mlc-ai/mlc-chat-WizardMath-13B-V1.0-q4f16_1

Cloning into 'mlc-chat-WizardMath-13B-V1.0-q4f16_1'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 173 (delta 0), reused 170 (delta 0), pack-reused 0
Receiving objects: 100% (173/173), 503.96 KiB | 2.95 MiB/s, done.
Filtering content: 100% (163/163), 6.82 GiB | 42.23 MiB/s, done.


In [ ]:
# In Colab, for some reason we need to restart runtime by running `exit()`.
# Simply run `exit()`, then run the subsequent cells after runtime restarts.
exit()

## Step 2. Let's Chat with WizardLM!

In [ ]:
from mlc_chat import ChatModule
from mlc_chat.callback import StreamToStdout

cm = ChatModule(model="WizardLM-13B-V1.2-q4f16_1", device="cuda")

Using model folder: /content/dist/prebuilt/mlc-chat-WizardLM-13B-V1.2-q4f16_1
Using mlc chat config: /content/dist/prebuilt/mlc-chat-WizardLM-13B-V1.2-q4f16_1/mlc-chat-config.json
Using library model: /content/dist/prebuilt/lib/Llama-2-13b-chat-hf-q4f16_1-cuda.so



In [ ]:
output = cm.generate(
    prompt="Give me three American cities names",
    progress_callback=StreamToStdout(callback_interval=2)
)

1. New York City
2. Los Angeles
3. Chicago


In [ ]:
output = cm.generate(
    prompt="Give me two more",
    progress_callback=StreamToStdout(callback_interval=2)
)

4. Las Vegas
5. Miami


In [ ]:
# In Colab, for some reason we need to restart runtime by running `exit()`.
# Simply run `exit()`, then run the subsequent cells after runtime restarts.
exit()

## Step 3. Let's Solve a Math Problem with WizardMath!

In [ ]:
from mlc_chat import ChatModule
from mlc_chat.callback import StreamToStdout

cm = ChatModule(model="WizardMath-13B-V1.0-q4f16_1", device="cuda")

Using model folder: /content/dist/prebuilt/mlc-chat-WizardMath-13B-V1.0-q4f16_1
Using mlc chat config: /content/dist/prebuilt/mlc-chat-WizardMath-13B-V1.0-q4f16_1/mlc-chat-config.json
Using library model: /content/dist/prebuilt/lib/WizardMath-13B-V1.0-q4f16_1-cuda.so



In [ ]:
prompt=(
    "A chicken has 2 legs, and a rabbit has 4 legs. Given that there are 5 animals in total, "
    "and 14 legs in total. How many chicken are there? Show your steps."
)
output = cm.generate(
    prompt=prompt,
    progress_callback=StreamToStdout(callback_interval=2)
)


Step 1: Define the variables.
Let C represent the number of chickens and R represent the number of rabbits.

Step 2: Write the equations based on the given information.
We know that the total number of legs is 14, so we can write the equation:
2C + 4R = 14

We also know that there are 5 animals in total, so we can write the equation:
C + R = 5

Step 3: Solve the system of equations.
We can solve this system of equations using the substitution method. First, we'll solve the second equation for R:
R = 5 - C

Now, we'll substitute this expression for R into the first equation:
2C + 4(5 - C) = 14

Step 4: Simplify and solve for C.
2C + 20 - 4C = 14
-2C = -6
C = 3

Step 5: Find the number of rabbits.
Now that we know there are 3 chickens, we can find the number of rabbits using the equation R = 5 - C:
R = 5 - 3
R = 2

Step 6: Provide the final answer.
There are 3 chickens and 2 rabbits, so the answer is:
C + R = 3 + 2 = 5
There are 5 animals in total, and since we found that there are 3 ch

In [ ]:
# In Colab, for some reason we need to restart runtime by running `exit()`.
# Simply run `exit()`, then run the subsequent cells after runtime restarts.
exit()

## Step 4. Let's Solve a Leetcode with WizardCoder!

WizardMath tends to give Markdown format output, which is really cool! We use `IPython.display` to display the output as Markdown!

In [ ]:
from mlc_chat import ChatModule
from mlc_chat.callback import StreamToStdout
from IPython.display import display, Markdown, Latex

cm = ChatModule(model="WizardCoder-15B-V1.0-q4f16_1", device="cuda")

Using model folder: /content/dist/prebuilt/mlc-chat-WizardCoder-15B-V1.0-q4f16_1
Using mlc chat config: /content/dist/prebuilt/mlc-chat-WizardCoder-15B-V1.0-q4f16_1/mlc-chat-config.json
Using library model: /content/dist/prebuilt/lib/WizardCoder-15B-V1.0-q4f16_1-cuda.so



In [ ]:
prompt= "Write a Python program that determines whether a number is a palindrome."

output = cm.generate(prompt=prompt)
print(output)

A palindrome is a number that reads the same backward as forward. For example, 12321 is a palindrome, while 12345 is not.

To determine whether a number is a palindrome, we can convert it to a string and compare the string with its reverse. If they are the same, then the number is a palindrome. Here's a Python program that does this:

```python
num = int(input("Enter a number: "))

# Convert the number to a string
num_str = str(num)

# Compare the string with its reverse
if num_str == num_str[::-1]:
    print(num, "is a palindrome")
else:
    print(num, "is not a palindrome")
```

In this program, we first prompt the user to enter a number. We then convert the number to a string using the `str()` function. We then compare the string with its reverse using slicing notation (`[::-1]`). If they are the same, then the number is a palindrome, and we print a message saying so. Otherwise, we print a message saying that the number is not a palindrome.


In [ ]:
display(Markdown(output))

A palindrome is a number that reads the same backward as forward. For example, 12321 is a palindrome, while 12345 is not.

To determine whether a number is a palindrome, we can convert it to a string and compare the string with its reverse. If they are the same, then the number is a palindrome. Here's a Python program that does this:

```python
num = int(input("Enter a number: "))

# Convert the number to a string
num_str = str(num)

# Compare the string with its reverse
if num_str == num_str[::-1]:
    print(num, "is a palindrome")
else:
    print(num, "is not a palindrome")
```

In this program, we first prompt the user to enter a number. We then convert the number to a string using the `str()` function. We then compare the string with its reverse using slicing notation (`[::-1]`). If they are the same, then the number is a palindrome, and we print a message saying so. Otherwise, we print a message saying that the number is not a palindrome.

In [ ]:
output = cm.generate(prompt="Write this in Java instead.")
print(output)

Here's the equivalent Java code:

```java
import java.util.Scanner;

public class Palindrome {
    public static void main(String[] args) {
        Scanner input = new Scanner(System.in);
        
        System.out.print("Enter a number: ");
        int num = input.nextInt();
        
        // Convert the number to a string
        String numStr = String.valueOf(num);
        
        // Compare the string with its reverse
        if (numStr.equals(new StringBuilder(numStr).reverse().toString())) {
            System.out.println(num + " is a palindrome");
        } else {
            System.out.println(num + " is not a palindrome");
        }
    }
}
```

In this code, we first prompt the user to enter a number using `System.in`. We then read the integer value from the input and store it in the `num` variable.

We then convert the number to a string using the `String.valueOf()` method. We then compare the string with its reverse using the `StringBuilder.reverse()` method and the `St

In [ ]:
display(Markdown(output))

Here's the equivalent Java code:

```java
import java.util.Scanner;

public class Palindrome {
    public static void main(String[] args) {
        Scanner input = new Scanner(System.in);
        
        System.out.print("Enter a number: ");
        int num = input.nextInt();
        
        // Convert the number to a string
        String numStr = String.valueOf(num);
        
        // Compare the string with its reverse
        if (numStr.equals(new StringBuilder(numStr).reverse().toString())) {
            System.out.println(num + " is a palindrome");
        } else {
            System.out.println(num + " is not a palindrome");
        }
    }
}
```

In this code, we first prompt the user to enter a number using `System.in`. We then read the integer value from the input and store it in the `num` variable.

We then convert the number to a string using the `String.valueOf()` method. We then compare the string with its reverse using the `StringBuilder.reverse()` method and the `String.equals()` method. If they are the same, then the number is a palindrome, and we print a message saying so. Otherwise, we print a message saying that the number is not a palindrome.